In [0]:
%fs rm -r /user/hive/warehouse

res1: Boolean = true

In [0]:

%sql
drop table if exists weather_log_table;
drop table if exists weather_raw;
drop table if exists weather_processed;
drop table if exists hourly_fact_weather;
drop table if exists daily_fact_weather;
drop table if exists dim_city;
drop table if exists dim_date;
drop table if exists dim_time;

### Running all the notebooks

##### Run the LogTable notebook

In [0]:
%run "/Users/nandansinghs007@gmail.com/LogTable"

##### Run the RawData

In [0]:
%run "/Users/nandansinghs007@gmail.com/RawData"

#### Run the processed data

In [0]:
%run "/Users/nandansinghs007@gmail.com/ProcessedData"

#### Running the dimTable notebook

In [0]:
%run "/Users/nandansinghs007@gmail.com/DimTables"

#### Loading data into fact table

In [0]:
%run "/Users/nandansinghs007@gmail.com/FactTables" 

### Main Pipeline

#### Getting List of cities and ID

In [0]:
cities = [('Wāliṅ',1282616),
          ('Upardang Gadhi',1282621),
          ('Tikoli',1282665),
          ('Ṭikāpur',1282666),
          ('Thapathali',1282682)
         ]

In [0]:
# Define the schema
schema = StructType([
    StructField("Name", StringType(), nullable=False),
    StructField("id", IntegerType(), nullable=False)
])


In [0]:
# Convert list to DataFrame
df = spark.createDataFrame(cities, schema)

In [0]:
#type casting id to integer
df = df.withColumn('id', col('id').cast('int'))

#### Getting raw and Processed Data

In [0]:
raw=get_raw_weather("Raw","weather_raw",df)


In [0]:
processed= get_process_weather("Processed","weather_Processed",raw)

In [0]:
display(processed)

dt,date,city_id,city_name,timezone,country,lat,lon,temp,temp_min,temp_max,pressure,humidity,visibility,wind_speed,wind_deg,wind_gust,clouds_all,load_run_id,created_on,created_by
1686444350,2023-06-11 00:45:50,1282616,Wāliṅ,20700,NP,27.9833,83.7667,300.74,300.74,300.74,1002,50,10000,null,273,1.08,null,9f35ab4e-21f6-4c50-808e-8159070d61a5,2023-06-11T00:46:15.858+0000,Nandan Singh
1686444346,2023-06-11 00:45:46,1282621,Upardang Gadhi,20700,NP,27.7667,84.5667,298.76,298.76,298.76,1001,49,10000,null,227,0.64,null,9f35ab4e-21f6-4c50-808e-8159070d61a5,2023-06-11T00:46:15.858+0000,Nandan Singh
1686444351,2023-06-11 00:45:51,1282665,Tikoli,20700,NP,27.6333,84.5,304.01,304.01,304.01,1000,49,10000,null,120,1.68,null,9f35ab4e-21f6-4c50-808e-8159070d61a5,2023-06-11T00:46:15.858+0000,Nandan Singh
1686444347,2023-06-11 00:45:47,1282666,Ṭikāpur,20700,NP,28.5,81.1333,304.79,304.79,304.79,998,26,10000,null,87,null,null,9f35ab4e-21f6-4c50-808e-8159070d61a5,2023-06-11T00:46:15.858+0000,Nandan Singh
1686444330,2023-06-11 00:45:30,1282682,Thapathali,20700,NP,27.7167,85.3167,296.39,295.27,296.39,1007,73,6000,null,200,null,null,9f35ab4e-21f6-4c50-808e-8159070d61a5,2023-06-11T00:46:15.858+0000,Nandan Singh


#### Calling the Dimension Table

In [0]:
load_date()
load_time()
load_city()

/databricks/spark/python/pyspark/pandas/namespace.py:1881: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  pd.date_range(


#### Calling the fact table

In [0]:
fact_df = get_hourly_fact_weather("FACT","fact_hourly_weather",processed)

In [0]:
daily_fact_df=get_daily_fact_weather("FACT","fact_daily_weather",processed)

#### Displaying the results of raw and processed data

In [0]:
%sql
select * from weather_raw

result,load_run_id,created_on,created_by
"List(10000, 20700, List(298.76, 298.66, 298.76, 298.76, 1001, 49, 1001, 887), List(null), List(NP, 1686439523, 1686489439), 1686444346, List(84.5667, 27.7667), Upardang Gadhi, List(List(802, Clouds, scattered clouds, 03d)), 200, 1282621, List(null, 227, 0.64), stations)",c08bcf1c-4315-4d67-a0f1-ccbc41b8dc2c,2023-06-11T00:45:34.980+0000,Nandan Singh
"List(10000, 20700, List(300.74, 301.15, 300.74, 300.74, 1002, 50, 1002, 917), List(null), List(NP, 1686439686, 1686489661), 1686444350, List(83.7667, 27.9833), Wāliṅ, List(List(800, Clear, clear sky, 01d)), 200, 1282616, List(null, 273, 1.08), stations)",c08bcf1c-4315-4d67-a0f1-ccbc41b8dc2c,2023-06-11T00:45:34.980+0000,Nandan Singh
"List(10000, 20700, List(304.01, 305.35, 304.01, 304.01, 1000, 49, 1000, 978), List(null), List(NP, 1686439558, 1686489437), 1686444351, List(84.5, 27.6333), Tikoli, List(List(801, Clouds, few clouds, 02d)), 200, 1282665, List(null, 120, 1.68), stations)",c08bcf1c-4315-4d67-a0f1-ccbc41b8dc2c,2023-06-11T00:45:34.980+0000,Nandan Singh
"List(10000, 20700, List(304.79, 303.19, 304.79, 304.79, 998, 26, 998, 981), List(null), List(NP, 1686440246, 1686490365), 1686444347, List(81.1333, 28.5), Ṭikāpur, List(List(800, Clear, clear sky, 01d)), 200, 1282666, List(null, 87, null), stations)",c08bcf1c-4315-4d67-a0f1-ccbc41b8dc2c,2023-06-11T00:45:34.980+0000,Nandan Singh
"List(6000, 20700, List(296.39, 296.68, 295.27, 296.39, 1007, 73, null, null), List(null), List(NP, 1686439350, 1686489253), 1686444350, List(85.3167, 27.7167), Thapathali, List(List(802, Clouds, scattered clouds, 03d)), 200, 1282682, List(null, 200, null), stations)",c08bcf1c-4315-4d67-a0f1-ccbc41b8dc2c,2023-06-11T00:45:34.980+0000,Nandan Singh


In [0]:
%sql
select * from weather_processed

dt,date,city_id,city_name,timezone,country,lat,lon,temp,temp_min,temp_max,pressure,humidity,visibility,wind_speed,wind_deg,wind_gust,clouds_all,load_run_id,created_on,created_by
1686444347,2023-06-11 00:45:47,1282621,Upardang Gadhi,20700,NP,27.7667,84.5667,298.76,298.76,298.76,1001,49,10000,null,227,0.64,null,9f35ab4e-21f6-4c50-808e-8159070d61a5,2023-06-11T00:46:15.858+0000,Nandan Singh
1686444346,2023-06-11 00:45:46,1282616,Wāliṅ,20700,NP,27.9833,83.7667,300.74,300.74,300.74,1002,50,10000,null,273,1.08,null,9f35ab4e-21f6-4c50-808e-8159070d61a5,2023-06-11T00:46:15.858+0000,Nandan Singh
1686444351,2023-06-11 00:45:51,1282665,Tikoli,20700,NP,27.6333,84.5,304.01,304.01,304.01,1000,49,10000,null,120,1.68,null,9f35ab4e-21f6-4c50-808e-8159070d61a5,2023-06-11T00:46:15.858+0000,Nandan Singh
1686444347,2023-06-11 00:45:47,1282666,Ṭikāpur,20700,NP,28.5,81.1333,304.79,304.79,304.79,998,26,10000,null,87,null,null,9f35ab4e-21f6-4c50-808e-8159070d61a5,2023-06-11T00:46:15.858+0000,Nandan Singh
1686444308,2023-06-11 00:45:08,1282682,Thapathali,20700,NP,27.7167,85.3167,296.39,295.27,296.39,1007,73,6000,null,200,null,null,9f35ab4e-21f6-4c50-808e-8159070d61a5,2023-06-11T00:46:15.858+0000,Nandan Singh


#### Dim table Results

In [0]:
%sql
select * from dim_date

dateID,fullDate,monthName,monthNumOfYear,dayNameOfWeek,dayNumOfWeek,dayNumOfMonth,dayNumOfYear,weekNumOfYear,quarterName,calenderQuarter,calenderYear
20230101,2023-01-01,January,1,Sunday,1,1,1,52,Q1,1,2023
20230102,2023-01-02,January,1,Monday,2,2,2,1,Q1,1,2023
20230103,2023-01-03,January,1,Tuesday,3,3,3,1,Q1,1,2023
20230104,2023-01-04,January,1,Wednesday,4,4,4,1,Q1,1,2023
20230105,2023-01-05,January,1,Thursday,5,5,5,1,Q1,1,2023
20230106,2023-01-06,January,1,Friday,6,6,6,1,Q1,1,2023
20230107,2023-01-07,January,1,Saturday,7,7,7,1,Q1,1,2023
20230108,2023-01-08,January,1,Sunday,1,8,8,1,Q1,1,2023
20230109,2023-01-09,January,1,Monday,2,9,9,2,Q1,1,2023
20230110,2023-01-10,January,1,Tuesday,3,10,10,2,Q1,1,2023


In [0]:
%sql
select * from dim_time

endTime,startTime,timeID
09:59,09:00,9
10:59,10:00,10
11:59,11:00,11
00:59,00:00,0
01:59,01:00,1
02:59,02:00,2
03:59,03:00,3
04:59,04:00,4
05:59,05:00,5
15:59,15:00,15


In [0]:
%sql
select * from dim_city

city_id,city_name,country,latitude,longitude,lat,lon
1282621,Upardang Gadhi,NP,null,null,27.7667,84.5667
1282616,Wāliṅ,NP,null,null,27.9833,83.7667
1282665,Tikoli,NP,null,null,27.6333,84.5
1282666,Ṭikāpur,NP,null,null,28.5,81.1333
1282682,Thapathali,NP,null,null,27.7167,85.3167


#### Fact table results

In [0]:
%sql
select * from fact_hourly_weather

timeID,dateID,city_id,temp,temp_min,temp_max,pressure,humidity,visibility,wind_speed,wind_deg,wind_gust,clouds_all,load_run_id,created_on,created_by
0,20230611,1282616,300.74,300.74,300.74,1002,50,10000,null,273,1.08,null,f2b9c22a-1bc1-46af-b140-a8e2f1c68398,2023-06-11T00:47:07.902+0000,Nandan Singh
0,20230611,1282621,298.76,298.76,298.76,1001,49,10000,null,227,0.64,null,f2b9c22a-1bc1-46af-b140-a8e2f1c68398,2023-06-11T00:47:07.902+0000,Nandan Singh
0,20230611,1282665,304.01,304.01,304.01,1000,49,10000,null,120,1.68,null,f2b9c22a-1bc1-46af-b140-a8e2f1c68398,2023-06-11T00:47:07.902+0000,Nandan Singh
0,20230611,1282666,304.79,304.79,304.79,998,26,10000,null,87,null,null,f2b9c22a-1bc1-46af-b140-a8e2f1c68398,2023-06-11T00:47:07.902+0000,Nandan Singh
0,20230611,1282682,296.39,295.27,296.39,1007,73,6000,null,200,null,null,f2b9c22a-1bc1-46af-b140-a8e2f1c68398,2023-06-11T00:47:07.902+0000,Nandan Singh


In [0]:
%sql
select * from fact_daily_weather

city_id,dateID,temp,temp_min,temp_max,pressure,humidity,visibility,wind_speed,wind_deg,wind_gust,clouds_all,load_run_id,created_on,created_by
1282616,20230611,300.739990234375,300.74,300.74,1002,50,10000,null,273,1.0800000429153442,null,b280db10-da89-4b78-8be8-ec70b5749036,2023-06-11T00:47:39.863+0000,Nandan Singh
1282621,20230611,298.760009765625,298.76,298.76,1001,49,10000,null,227,0.6399999856948853,null,b280db10-da89-4b78-8be8-ec70b5749036,2023-06-11T00:47:39.863+0000,Nandan Singh
1282665,20230611,304.010009765625,304.01,304.01,1000,49,10000,null,120,1.6799999475479126,null,b280db10-da89-4b78-8be8-ec70b5749036,2023-06-11T00:47:39.863+0000,Nandan Singh
1282666,20230611,304.7900085449219,304.79,304.79,998,26,10000,null,87,null,null,b280db10-da89-4b78-8be8-ec70b5749036,2023-06-11T00:47:39.863+0000,Nandan Singh
1282682,20230611,296.3900146484375,295.27,296.39,1007,73,6000,null,200,null,null,b280db10-da89-4b78-8be8-ec70b5749036,2023-06-11T00:47:39.863+0000,Nandan Singh


In [0]:
%sql
select * from weather_log_table

id,load_type,table_name,process_start_time,process_end_time,status,comments,start_date_time,end_date_time,created_on,created_by
9f35ab4e-21f6-4c50-808e-8159070d61a5,Processed,weather_Processed,2023-06-11T00:46:15.858+0000,2023-06-11T00:46:32.872+0000,COMPLETED,null,2023-06-11T00:45:30.000+0000,2023-06-11T00:46:22.000+0000,2023-06-11T00:46:15.858+0000,Nandan Singh
f2b9c22a-1bc1-46af-b140-a8e2f1c68398,FACT,fact_hourly_weather,2023-06-11T00:47:07.902+0000,2023-06-11T00:47:30.850+0000,COMPLETED,null,2023-06-11T00:45:46.000+0000,2023-06-11T00:45:51.000+0000,2023-06-11T00:47:07.902+0000,Nandan Singh
b280db10-da89-4b78-8be8-ec70b5749036,FACT,fact_daily_weather,2023-06-11T00:47:39.863+0000,2023-06-11T00:48:05.694+0000,COMPLETED,null,2023-06-11T00:45:08.000+0000,2023-06-11T00:45:51.000+0000,2023-06-11T00:47:39.863+0000,Nandan Singh
c08bcf1c-4315-4d67-a0f1-ccbc41b8dc2c,Raw,weather_raw,2023-06-11T00:45:34.980+0000,2023-06-11T00:45:59.640+0000,COMPLETED,null,2023-06-11T00:45:30.000+0000,2023-06-11T00:45:47.000+0000,2023-06-11T00:45:34.980+0000,Nandan Singh


### Running Forcasting Notebook

In [0]:
%run "/Users/nandansinghs007@gmail.com/WeatherForecasting"

##### Calling the weather forcasted function

In [0]:
forcasted_df = get_forcasted_weather("forcasted_data","forcasted_fact_hourly",processed)

In [0]:
display(forcasted_df)

timeID,dateID,city_id,temp,temp_min,temp_max,pressure,humidity,visibility,wind_speed,wind_deg,wind_gust,clouds_all,isForecasted,load_run_id,created_on,created_by
0,20230611,1282616,300.739990234375,300.74,300.74,1002,50,10000,null,273,1.0800000429153442,null,No,7c3b9f03-67f9-4534-bcb2-d6d8f7cba26a,2023-06-11T01:20:56.624+0000,Nandan Singh
0,20230611,1282621,298.760009765625,298.76,298.76,1001,49,10000,null,227,0.6399999856948853,null,No,7c3b9f03-67f9-4534-bcb2-d6d8f7cba26a,2023-06-11T01:20:56.624+0000,Nandan Singh
0,20230611,1282665,304.010009765625,304.01,304.01,1000,49,10000,null,120,1.6799999475479126,null,No,7c3b9f03-67f9-4534-bcb2-d6d8f7cba26a,2023-06-11T01:20:56.624+0000,Nandan Singh
0,20230611,1282666,304.7900085449219,304.79,304.79,998,26,10000,null,87,null,null,No,7c3b9f03-67f9-4534-bcb2-d6d8f7cba26a,2023-06-11T01:20:56.624+0000,Nandan Singh
0,20230611,1282682,296.3900146484375,296.27,296.39,1007,73,6000,null,200,null,null,No,7c3b9f03-67f9-4534-bcb2-d6d8f7cba26a,2023-06-11T01:20:56.624+0000,Nandan Singh
1,20230611,1282616,300.739990234375,300.74,300.74,1002,50,10000,null,273,1.0800000429153442,null,Yes,7c3b9f03-67f9-4534-bcb2-d6d8f7cba26a,2023-06-11T01:20:56.624+0000,Nandan Singh
1,20230611,1282621,298.760009765625,298.76,298.76,1001,49,10000,null,227,0.6399999856948853,null,Yes,7c3b9f03-67f9-4534-bcb2-d6d8f7cba26a,2023-06-11T01:20:56.624+0000,Nandan Singh
1,20230611,1282665,304.010009765625,304.01,304.01,1000,49,10000,null,120,1.6799999475479126,null,Yes,7c3b9f03-67f9-4534-bcb2-d6d8f7cba26a,2023-06-11T01:20:56.624+0000,Nandan Singh
1,20230611,1282666,304.7900085449219,304.79,304.79,998,26,10000,null,87,null,null,Yes,7c3b9f03-67f9-4534-bcb2-d6d8f7cba26a,2023-06-11T01:20:56.624+0000,Nandan Singh
1,20230611,1282682,296.3900146484375,296.27,296.39,1007,73,6000,null,200,null,null,Yes,7c3b9f03-67f9-4534-bcb2-d6d8f7cba26a,2023-06-11T01:20:56.624+0000,Nandan Singh


In [0]:
%sql
select * from weather_log_table

id,load_type,table_name,process_start_time,process_end_time,status,comments,start_date_time,end_date_time,created_on,created_by
ac846dac-699f-4d96-a4b0-1ea0d2b51602,forcasted_data,"DataFrame[timeID: int, dateID: int, city_id: int, temp: float, temp_min: float, temp_max: float, pressure: int, humidity: int, visibility: int, wind_speed: int, wind_deg: int, wind_gust: float, clouds_all: float, load_run_id: string, created_on: timestamp, created_by: string]",2023-06-11T01:13:07.897+0000,null,EXTRACTING,null,null,null,2023-06-11T01:13:07.897+0000,Nandan Singh
6b00247e-29a9-4415-b491-171f599be821,forcasted_data,"DataFrame[timeID: int, dateID: int, city_id: int, temp: float, temp_min: float, temp_max: float, pressure: int, humidity: int, visibility: int, wind_speed: int, wind_deg: int, wind_gust: float, clouds_all: float, load_run_id: string, created_on: timestamp, created_by: string]",2023-06-11T01:15:49.851+0000,null,EXTRACTING,null,null,null,2023-06-11T01:15:49.851+0000,Nandan Singh
7c3b9f03-67f9-4534-bcb2-d6d8f7cba26a,forcasted_data,forcasted_fact_hourly,2023-06-11T01:20:56.624+0000,2023-06-11T01:21:39.575+0000,COMPLETED,null,2023-06-11T01:17:06.000+0000,2023-06-11T01:21:08.000+0000,2023-06-11T01:20:56.624+0000,Nandan Singh
9f35ab4e-21f6-4c50-808e-8159070d61a5,Processed,weather_Processed,2023-06-11T00:46:15.858+0000,2023-06-11T00:46:32.872+0000,COMPLETED,null,2023-06-11T00:45:30.000+0000,2023-06-11T00:46:22.000+0000,2023-06-11T00:46:15.858+0000,Nandan Singh
f2b9c22a-1bc1-46af-b140-a8e2f1c68398,FACT,fact_hourly_weather,2023-06-11T00:47:07.902+0000,2023-06-11T00:47:30.850+0000,COMPLETED,null,2023-06-11T00:45:46.000+0000,2023-06-11T00:45:51.000+0000,2023-06-11T00:47:07.902+0000,Nandan Singh
b280db10-da89-4b78-8be8-ec70b5749036,FACT,fact_daily_weather,2023-06-11T00:47:39.863+0000,2023-06-11T00:48:05.694+0000,COMPLETED,null,2023-06-11T00:45:08.000+0000,2023-06-11T00:45:51.000+0000,2023-06-11T00:47:39.863+0000,Nandan Singh
c08bcf1c-4315-4d67-a0f1-ccbc41b8dc2c,Raw,weather_raw,2023-06-11T00:45:34.980+0000,2023-06-11T00:45:59.640+0000,COMPLETED,null,2023-06-11T00:45:30.000+0000,2023-06-11T00:45:47.000+0000,2023-06-11T00:45:34.980+0000,Nandan Singh
7fa025ec-1dab-4a44-accb-122d3481c37d,forcasted_data,forcasted_fact_hourly,2023-06-11T01:16:38.061+0000,null,EXTRACTING,null,null,null,2023-06-11T01:16:38.061+0000,Nandan Singh
